<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [11]:
from planet4 import region_data, io, catalog_production
import pvl
from hirise_tools import products, downloads, labels
from osgeo import gdal

In [2]:
regions = ['Inca', 'Ithaca', 'Giza', 'Manhattan2', 'Starburst', 'Oswego_Edge',
           'Maccelsfield', 'BuenosAires', 'Potsdam']

In [3]:
seasons = ['season2', 'season3']

In [4]:
def get_fraction_of_black_pixels(savepath):
    ds = gdal.Open(str(savepath))
    data = ds.ReadAsArray()
    fractions = []
    for band in data:
        nonzeros = band.nonzero()[0]
        fractions.append((band.size - nonzeros.size)/band.size)
    return np.array(fractions).mean()

In [5]:
def get_labelpath(obsid):
    prodid = products.PRODUCT_ID(obsid)
    prodid.kind = 'COLOR'
    labelpath = downloads.labels_root() / prodid.label_fname
    return labelpath

def get_p4_hirise_label(obsid):
    labelpath = get_labelpath(obsid)
    if not labelpath.exists():
        downloads.get_rdr_color_label(obsid)
    return labels.HiRISE_Label(labelpath)    

def calc_real_area(savepath):
    black_fraction = get_fraction_of_black_pixels(savepath)
    all_area = label.line_samples*label.lines * label.map_scale**2
    real_area = (1-black_fraction)*all_area
    return real_area

def read_metadata(obsid):
    label = get_p4_hirise_label(obsid)
    d = dict(obsid=obsid, binning=label.binning,
             l_s=label.l_s, line_samples=label.line_samples,
             lines=label.lines, map_scale=label.map_scale)
#     invalids=black_fraction, real_area=real_area)
    # self calculated north azimuths
#     folder = io.get_ground_projection_root()
#     path = folder / obsid / f"{obsid}_campt_out.csv"
#     df = pd.read_csv(path)
#     d['north_azimuth'] = df.NorthAzimuth.median()
    return d

In [6]:
obsid = 'ESP_011422_0930'

In [7]:
lbl = get_p4_hirise_label(obsid)

In [8]:
from planetpy.pdstools import indices

In [27]:
lbl = indices.IndexLabel("/Volumes/Data/hirise/EDRCUMINDEX.LBL")

In [139]:
edrindex = pd.read_hdf("/Volumes/Data/hirise/EDRCUMINDEX.hdf")

In [17]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [169]:
rm = catalog_production.ReleaseManager('v1.0')

In [15]:
obsids = rm.obsids

In [154]:
p4_edr = edrindex[edrindex.OBSERVATION_ID.isin(obsids)].query('CCD_NAME=="RED4"').drop_duplicates(subset='OBSERVATION_ID')

In [155]:
p4_edr.LINE_SAMPLES.value_counts()

512     127
1024     75
256      19
Name: LINE_SAMPLES, dtype: int64

In [156]:
p4_edr.BINNING.value_counts()

2    127
1     75
4     19
Name: BINNING, dtype: int64

In [157]:
[i for i in p4_edr.columns if 'SCALE' in i]

['SCALED_PIXEL_WIDTH']

In [98]:
p4_edr.SCALED_PIXEL_WIDTH.round(decimals=1).value_counts()

0.5    126
0.2     73
1.0     19
0.3      2
0.6      1
Name: SCALED_PIXEL_WIDTH, dtype: int64

In [99]:
%matplotlib ipympl

In [105]:
plt.figure()
p4_edr.query('BINNING==2').SCALED_PIXEL_WIDTH.hist(bins=50)

FigureCanvasNbAgg()

In [158]:
from planet4 import metadata

In [159]:
NAs = metadata.get_north_azimuths_from_SPICE(obsids)

In [160]:
NAs.head()

,OBSERVATION_ID,north_azimuth
0,ESP_012079_0945,126.856883
1,ESP_011697_0980,109.322541
2,ESP_021455_0935,138.529846
3,ESP_012008_0975,109.471252
4,ESP_012436_0980,109.045590


In [161]:
p4_edr = p4_edr.set_index('OBSERVATION_ID').join(NAs.set_index('OBSERVATION_ID'))

In [166]:
p4_edr.head()

,VOLUME_ID,FILE_NAME_SPECIFICATION,INSTRUMENT_HOST_ID,INSTRUMENT_ID,PRODUCT_ID,PRODUCT_VERSION_ID,HICAL_VERSION,TARGET_NAME,ORBIT_NUMBER,MISSION_PHASE_NAME,RATIONALE_DESC,OBSERVATION_START_TIME,OBSERVATION_START_COUNT,START_TIME,SPACECRAFT_CLOCK_START_COUNT,STOP_TIME,SPACECRAFT_CLOCK_STOP_COUNT,CCD_NAME,CHANNEL_NUMBER,FILTER_NAME,SCAN_EXPOSURE_DURATION,DELTA_LINE_TIME_COUNT,BINNING,TDI,TRIM_LINES,FOCUS_POSITION_COUNT,FELICS_COMPRESSION_FLAG,STIMULATION_LAMP_FLAG_RED,STIMULATION_LAMP_FLAG_BLUEGREEN,STIMULATION_LAMP_FLAG_NEARINFRARED,LOOKUP_TABLE_TYPE,LOOKUP_TABLE_MINIMUM,LOOKUP_TABLE_MAXIMUM,LOOKUP_TABLE_MEDIAN,LOOKUP_TABLE_K_VALUE,LOOKUP_TABLE_NUMBER,ADC_CONVERSION_SETTINGS_1,ADC_CONVERSION_SETTINGS_2,FPA_POSITIVE_Y_TEMPERATURE,FPA_NEGATIVE_Y_TEMPERATURE,FPE_TEMPERATURE,IEA_TEMPERATURE,IEA_PWS_BOARD_TEMPERATURE,CPMM_PWS_BOARD_TEMPERATURE,IMAGE_LINES,LINE_SAMPLES,SAMPLE_BITS,SCALED_PIXEL_WIDTH,EMISSION_ANGLE,INCIDENCE_ANGLE,PHASE_ANGLE,IMAGE_CENTER_LATITUDE,IMAGE_CENTER_LONGITUDE,MINIMUM_LATITUDE,MAXIMUM_LATITUDE,MINIMUM_LONGITUDE,MAXIMUM_LONGITUDE,SPACECRAFT_ALTITUDE,TARGET_CENTER_DISTANCE,SLANT_DISTANCE,NORTH_AZIMUTH,SUB_SOLAR_AZIMUTH,SUB_SOLAR_LATITUDE,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,STEREO_FLAG,north_azimuth,image_id
OBSERVATION_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ESP_011296_0975,MROHR_0001,EDR/ESP/ORB_011200_011299/ESP_011296_0975/ESP_...,MRO,HIRISE,ESP_011296_0975_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11296,Extended Science Phase,Fans in clusters,2008-12-23 16:15:26,914516147:11317,2008-12-23 16:15:26,914516147:25170,2008-12-23 16:15:42,914516162:51385,RED4,0,RED,77.0000,48,4,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,11,5,4,33.7490,33.4008,34.0103,27.0515,37.5539,47.9183,50000,256,8,0.993332,0.14182,88.7497,88.6868,-82.1965,225.2530,-82.6044,-81.7872,223.9370,226.6930,248.229,3629.42,248.230,110.312,33.0075,0.4965,148.2010,-82.2040,225.2080,1.46848,178.833,17.1368,NO,110.600107,91
ESP_011341_0980,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011341_0980/ESP_...,MRO,HIRISE,ESP_011341_0980_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11341,Extended Science Phase,Starburst araneiform,2008-12-27 04:25:02,914819122:46720,2008-12-27 04:25:02,914819122:58478,2008-12-27 04:25:07,914819128:16207,RED4,0,RED,76.5000,40,2,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,13,5,4,32.8784,32.5303,33.4008,26.9646,37.4400,47.9183,35000,512,8,0.493999,0.25322,87.7623,88.0088,-81.7969,76.1304,-81.9487,-81.6449,75.5112,76.7652,246.898,3627.50,246.897,109.702,32.2540,-0.3443,359.5500,-81.7969,76.2512,1.46414,180.809,17.1054,NO,110.208923,126
ESP_011348_0950,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011348_0950/ESP_...,MRO,HIRISE,ESP_011348_0950_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11348,Extended Science Phase,Polar araneiform trough network,2008-12-27 17:29:17,914866178:01633,2008-12-27 17:29:17,914866178:15487,2008-12-27 17:29:33,914866193:41701,RED4,0,RED,77.0000,48,4,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,12,5,4,32.5303,32.2692,33.1396,26.9646,36.9845,47.4627,50000,256,8,0.993426,0.15704,89.5609,89.7062,-85.0427,259.0940,-85.4128,-84.6651,256.1920,262.3920,248.258,3629.01,248.253,123.123,32.6524,-0.4754,168.7050,-85.0385,259.2080,1.46347,181.117,18.0259,NO,123.624057,91
ESP_011350_0945,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011350_0945/ESP_...,MRO,HIRISE,ESP_011350_0945_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11350,Extended Science Phase,Ithaca region plumes,2008-12-27 21:14:00,914879661:30743,2008-12-27 21:14:01,914879661:43643,2008-12-27 21:14:07,914879667:35492,RED4,0,RED,83.9375,159,2,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,13,5,4,32.7044,32.3562,33.2267,26.9646,36.9845,47.3488,35000,512,8,0.539147,24.58710,87.6506,108.1410,-85.2160,181.4150,-85.3824,-85.0496,180.4970,182.3690,246.792,3628.33,269.461,99.362,34.6041,-0.5129,1

In [115]:
db = io.DBManager()

In [116]:
all_data = db.get_all()

In [118]:
no_of_tiles_per_obsid = all_data.groupby('image_name').image_id.nunique()

In [163]:
p4_edr = p4_edr.join(no_of_tiles_per_obsid)

In [164]:
p4_edr.head()

,VOLUME_ID,FILE_NAME_SPECIFICATION,INSTRUMENT_HOST_ID,INSTRUMENT_ID,PRODUCT_ID,PRODUCT_VERSION_ID,HICAL_VERSION,TARGET_NAME,ORBIT_NUMBER,MISSION_PHASE_NAME,RATIONALE_DESC,OBSERVATION_START_TIME,OBSERVATION_START_COUNT,START_TIME,SPACECRAFT_CLOCK_START_COUNT,STOP_TIME,SPACECRAFT_CLOCK_STOP_COUNT,CCD_NAME,CHANNEL_NUMBER,FILTER_NAME,SCAN_EXPOSURE_DURATION,DELTA_LINE_TIME_COUNT,BINNING,TDI,TRIM_LINES,FOCUS_POSITION_COUNT,FELICS_COMPRESSION_FLAG,STIMULATION_LAMP_FLAG_RED,STIMULATION_LAMP_FLAG_BLUEGREEN,STIMULATION_LAMP_FLAG_NEARINFRARED,LOOKUP_TABLE_TYPE,LOOKUP_TABLE_MINIMUM,LOOKUP_TABLE_MAXIMUM,LOOKUP_TABLE_MEDIAN,LOOKUP_TABLE_K_VALUE,LOOKUP_TABLE_NUMBER,ADC_CONVERSION_SETTINGS_1,ADC_CONVERSION_SETTINGS_2,FPA_POSITIVE_Y_TEMPERATURE,FPA_NEGATIVE_Y_TEMPERATURE,FPE_TEMPERATURE,IEA_TEMPERATURE,IEA_PWS_BOARD_TEMPERATURE,CPMM_PWS_BOARD_TEMPERATURE,IMAGE_LINES,LINE_SAMPLES,SAMPLE_BITS,SCALED_PIXEL_WIDTH,EMISSION_ANGLE,INCIDENCE_ANGLE,PHASE_ANGLE,IMAGE_CENTER_LATITUDE,IMAGE_CENTER_LONGITUDE,MINIMUM_LATITUDE,MAXIMUM_LATITUDE,MINIMUM_LONGITUDE,MAXIMUM_LONGITUDE,SPACECRAFT_ALTITUDE,TARGET_CENTER_DISTANCE,SLANT_DISTANCE,NORTH_AZIMUTH,SUB_SOLAR_AZIMUTH,SUB_SOLAR_LATITUDE,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,STEREO_FLAG,north_azimuth,image_id
OBSERVATION_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ESP_011296_0975,MROHR_0001,EDR/ESP/ORB_011200_011299/ESP_011296_0975/ESP_...,MRO,HIRISE,ESP_011296_0975_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11296,Extended Science Phase,Fans in clusters,2008-12-23 16:15:26,914516147:11317,2008-12-23 16:15:26,914516147:25170,2008-12-23 16:15:42,914516162:51385,RED4,0,RED,77.0000,48,4,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,11,5,4,33.7490,33.4008,34.0103,27.0515,37.5539,47.9183,50000,256,8,0.993332,0.14182,88.7497,88.6868,-82.1965,225.2530,-82.6044,-81.7872,223.9370,226.6930,248.229,3629.42,248.230,110.312,33.0075,0.4965,148.2010,-82.2040,225.2080,1.46848,178.833,17.1368,NO,110.600107,91
ESP_011341_0980,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011341_0980/ESP_...,MRO,HIRISE,ESP_011341_0980_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11341,Extended Science Phase,Starburst araneiform,2008-12-27 04:25:02,914819122:46720,2008-12-27 04:25:02,914819122:58478,2008-12-27 04:25:07,914819128:16207,RED4,0,RED,76.5000,40,2,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,13,5,4,32.8784,32.5303,33.4008,26.9646,37.4400,47.9183,35000,512,8,0.493999,0.25322,87.7623,88.0088,-81.7969,76.1304,-81.9487,-81.6449,75.5112,76.7652,246.898,3627.50,246.897,109.702,32.2540,-0.3443,359.5500,-81.7969,76.2512,1.46414,180.809,17.1054,NO,110.208923,126
ESP_011348_0950,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011348_0950/ESP_...,MRO,HIRISE,ESP_011348_0950_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11348,Extended Science Phase,Polar araneiform trough network,2008-12-27 17:29:17,914866178:01633,2008-12-27 17:29:17,914866178:15487,2008-12-27 17:29:33,914866193:41701,RED4,0,RED,77.0000,48,4,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,12,5,4,32.5303,32.2692,33.1396,26.9646,36.9845,47.4627,50000,256,8,0.993426,0.15704,89.5609,89.7062,-85.0427,259.0940,-85.4128,-84.6651,256.1920,262.3920,248.258,3629.01,248.253,123.123,32.6524,-0.4754,168.7050,-85.0385,259.2080,1.46347,181.117,18.0259,NO,123.624057,91
ESP_011350_0945,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011350_0945/ESP_...,MRO,HIRISE,ESP_011350_0945_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11350,Extended Science Phase,Ithaca region plumes,2008-12-27 21:14:00,914879661:30743,2008-12-27 21:14:01,914879661:43643,2008-12-27 21:14:07,914879667:35492,RED4,0,RED,83.9375,159,2,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,13,5,4,32.7044,32.3562,33.2267,26.9646,36.9845,47.3488,35000,512,8,0.539147,24.58710,87.6506,108.1410,-85.2160,181.4150,-85.3824,-85.0496,180.4970,182.3690,246.792,3628.33,269.461,99.362,34.6041,-0.5129,1

In [167]:
p4_edr.rename(dict(image_id="# of tiles"), axis=1,inplace=True)

In [168]:
p4_edr.head()

,VOLUME_ID,FILE_NAME_SPECIFICATION,INSTRUMENT_HOST_ID,INSTRUMENT_ID,PRODUCT_ID,PRODUCT_VERSION_ID,HICAL_VERSION,TARGET_NAME,ORBIT_NUMBER,MISSION_PHASE_NAME,RATIONALE_DESC,OBSERVATION_START_TIME,OBSERVATION_START_COUNT,START_TIME,SPACECRAFT_CLOCK_START_COUNT,STOP_TIME,SPACECRAFT_CLOCK_STOP_COUNT,CCD_NAME,CHANNEL_NUMBER,FILTER_NAME,SCAN_EXPOSURE_DURATION,DELTA_LINE_TIME_COUNT,BINNING,TDI,TRIM_LINES,FOCUS_POSITION_COUNT,FELICS_COMPRESSION_FLAG,STIMULATION_LAMP_FLAG_RED,STIMULATION_LAMP_FLAG_BLUEGREEN,STIMULATION_LAMP_FLAG_NEARINFRARED,LOOKUP_TABLE_TYPE,LOOKUP_TABLE_MINIMUM,LOOKUP_TABLE_MAXIMUM,LOOKUP_TABLE_MEDIAN,LOOKUP_TABLE_K_VALUE,LOOKUP_TABLE_NUMBER,ADC_CONVERSION_SETTINGS_1,ADC_CONVERSION_SETTINGS_2,FPA_POSITIVE_Y_TEMPERATURE,FPA_NEGATIVE_Y_TEMPERATURE,FPE_TEMPERATURE,IEA_TEMPERATURE,IEA_PWS_BOARD_TEMPERATURE,CPMM_PWS_BOARD_TEMPERATURE,IMAGE_LINES,LINE_SAMPLES,SAMPLE_BITS,SCALED_PIXEL_WIDTH,EMISSION_ANGLE,INCIDENCE_ANGLE,PHASE_ANGLE,IMAGE_CENTER_LATITUDE,IMAGE_CENTER_LONGITUDE,MINIMUM_LATITUDE,MAXIMUM_LATITUDE,MINIMUM_LONGITUDE,MAXIMUM_LONGITUDE,SPACECRAFT_ALTITUDE,TARGET_CENTER_DISTANCE,SLANT_DISTANCE,NORTH_AZIMUTH,SUB_SOLAR_AZIMUTH,SUB_SOLAR_LATITUDE,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,STEREO_FLAG,north_azimuth,# of tiles
OBSERVATION_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ESP_011296_0975,MROHR_0001,EDR/ESP/ORB_011200_011299/ESP_011296_0975/ESP_...,MRO,HIRISE,ESP_011296_0975_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11296,Extended Science Phase,Fans in clusters,2008-12-23 16:15:26,914516147:11317,2008-12-23 16:15:26,914516147:25170,2008-12-23 16:15:42,914516162:51385,RED4,0,RED,77.0000,48,4,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,11,5,4,33.7490,33.4008,34.0103,27.0515,37.5539,47.9183,50000,256,8,0.993332,0.14182,88.7497,88.6868,-82.1965,225.2530,-82.6044,-81.7872,223.9370,226.6930,248.229,3629.42,248.230,110.312,33.0075,0.4965,148.2010,-82.2040,225.2080,1.46848,178.833,17.1368,NO,110.600107,91
ESP_011341_0980,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011341_0980/ESP_...,MRO,HIRISE,ESP_011341_0980_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11341,Extended Science Phase,Starburst araneiform,2008-12-27 04:25:02,914819122:46720,2008-12-27 04:25:02,914819122:58478,2008-12-27 04:25:07,914819128:16207,RED4,0,RED,76.5000,40,2,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,13,5,4,32.8784,32.5303,33.4008,26.9646,37.4400,47.9183,35000,512,8,0.493999,0.25322,87.7623,88.0088,-81.7969,76.1304,-81.9487,-81.6449,75.5112,76.7652,246.898,3627.50,246.897,109.702,32.2540,-0.3443,359.5500,-81.7969,76.2512,1.46414,180.809,17.1054,NO,110.208923,126
ESP_011348_0950,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011348_0950/ESP_...,MRO,HIRISE,ESP_011348_0950_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11348,Extended Science Phase,Polar araneiform trough network,2008-12-27 17:29:17,914866178:01633,2008-12-27 17:29:17,914866178:15487,2008-12-27 17:29:33,914866193:41701,RED4,0,RED,77.0000,48,4,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,12,5,4,32.5303,32.2692,33.1396,26.9646,36.9845,47.4627,50000,256,8,0.993426,0.15704,89.5609,89.7062,-85.0427,259.0940,-85.4128,-84.6651,256.1920,262.3920,248.258,3629.01,248.253,123.123,32.6524,-0.4754,168.7050,-85.0385,259.2080,1.46347,181.117,18.0259,NO,123.624057,91
ESP_011350_0945,MROHR_0001,EDR/ESP/ORB_011300_011399/ESP_011350_0945/ESP_...,MRO,HIRISE,ESP_011350_0945_RED4_0,1,3.1.20 | 2009-06-10 | 2008-11-17,MARS,11350,Extended Science Phase,Ithaca region plumes,2008-12-27 21:14:00,914879661:30743,2008-12-27 21:14:01,914879661:43643,2008-12-27 21:14:07,914879667:35492,RED4,0,RED,83.9375,159,2,128,1797,2023,YES,OFF,OFF,OFF,STORED,-9998,-9998,-9998,-9998,13,5,4,32.7044,32.3562,33.2267,26.9646,36.9845,47.3488,35000,512,8,0.539147,24.58710,87.6506,108.1410,-85.2160,181.4150,-85.3824,-85.0496,180.4970,182.3690,246.792,3628.33,269.461,99.362,34.6041,-0.5129

In [170]:
p4_edr.to_csv(rm.EDRINDEX_meta_path)

In [ ]:
done_obsids = pd.read_csv("/Users/klay6683/local_data/planet4/done_obsids_n_tiles.csv",
                          index_col=0)

In [ ]:
done_obsids.head()

In [ ]:
obsids = done_obsids.image_name.unique()

In [ ]:
%matplotlib ipympl

In [ ]:
done_obsids.groupby('image_name').size().hist(bins=50)

In [ ]:
mr = MetadataReader(obsids[0])

In [ ]:
mr.data_dic

In [ ]:
read_metadata(obsids[0])

In [ ]:
from pathlib import Path

In [ ]:
catalog = 'catalog_1.0b3'

In [ ]:
from tqdm import tqdm
metadata = []
for img in tqdm(obsids):
    metadata.append(read_metadata(img))
df = pd.DataFrame(metadata)

In [ ]:
df.head()

In [ ]:
savefolder = io.data_root / catalog

In [ ]:
savepath = savefolder / "all_metadata.csv"

In [ ]:
savepath

In [ ]:
list(savefolder.glob("*.csv"))

In [ ]:
df.to_csv(savepath, index=False)

In [ ]:
    
    name = "{}_metadata.csv".format(region.lower())
    folder = io.analysis_folder() / catalog / region.lower()
    folder.mkdir(exist_ok=True, parents=True)
    fname = folder / name
    pd.concat(bucket, ignore_index=True).to_csv(str(fname), index=False)
    print("Created", fname)